### Modules to import

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import onnxruntime as ort
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

In [5]:
from functions.utilities import process_output_data, load_and_stack_processed_images
from functions.data_processing import process_files, prepare_train_test_dataloaders
from functions.model import TimeSeriesToImageModelCont
from functions.train_validate import train_and_validate, get_input_sample, load_model_onnx
from functions.evaluate import evaluate_model_onnx
from functions.hp_tuning import run_optuna_study
from functions.metrics import (
    calculate_blob_extents,
    calculate_areas,
    calculate_mape,
    compute_angles,
    compute_centroids,
    compute_centroid_distances,
    calculate_nrmse,
    calculate_nmi,
    calculate_ssim,
    determine_dynamic_roi,
)

### Data loading and processing

In [6]:
#Set paths and parameters
curr_path=os.getcwd()
path_data=os.path.join(curr_path, "Results")
path_output=os.path.join(curr_path, "wearscar_processed_100x100")
inputs=["ap", "fe", "force", "ie"]
output="wearscar"
m=100
n=100
plot=True
save=True
save_suffix="wearscar_processed_100x100"
file_suffix=f"{m}x{n}"

#Process input and output files
all_inputs, all_output, columns_output=process_files(path_data, inputs, output)

all_input_ap=all_inputs["ap"]
all_input_fe=all_inputs["fe"]
all_input_force=all_inputs["force"]
all_input_ie=all_inputs["ie"]

#Process and save the output data
process_output_data(columns_output, path_data, save_suffix, m, n, plot, save)

#Load and stack processed images
all_data_output=load_and_stack_processed_images(path_output, m, n, file_suffix)

### Model training 

In [9]:
#Prepare DataLoaders for training and testing
train_dataloader, test_dataloader=prepare_train_test_dataloaders(
    all_input_ap, all_input_fe, all_input_force, all_input_ie,
    all_data_output, inputs, test_size=0.2, random_state=15, batch_size=4
)


#Set up the model
model=TimeSeriesToImageModelCont()
optimizer=optim.Adam(model.parameters(), lr=0.00001)
criterion=nn.MSELoss()
num_epochs=1000
onnx_checkpoint_path='best_model_g2c.onnx'

input_sample=get_input_sample(train_dataloader, batch_size=4)

#Train and validate the model, saving the best model as ONNX
train_losses, val_losses=train_and_validate(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    num_epochs=num_epochs,
    optimizer=optimizer,
    criterion=criterion,
    checkpoint_path=onnx_checkpoint_path,
    input_sample=input_sample,
    plot_every=10
)


C:\Users\Mattia\anaconda3\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


### Inference and computing metrics

In [10]:
#Evaluating the model on test data
onnx_session=load_model_onnx(onnx_checkpoint_path)
predictions_test, ground_truth_test=evaluate_model_onnx(
    session=onnx_session, 
    dataloader=test_dataloader, 
    threshold=5e-4
)

#Parameters
image_shape=predictions_test.shape[2:]
thresh_zero=5e-4

#Calculate blob extents
x_ext_pred, y_ext_pred, x_ext_ground, y_ext_ground=calculate_blob_extents(predictions_test, ground_truth_test, thresh_zero)

#Calculate MAPE for extents
mape_x_first=calculate_mape(x_ext_ground['first'], x_ext_pred['first'])
mape_x_second=calculate_mape(x_ext_ground['second'], x_ext_pred['second'])
mape_y_first=calculate_mape(y_ext_ground['first'], y_ext_pred['first'])
mape_y_second=calculate_mape(y_ext_ground['second'], y_ext_pred['second'])

#Calculate areas
areas_pred, areas_ground=calculate_areas(predictions_test, ground_truth_test)

#Calculate MAPE for areas
mape_non_weight_first=calculate_mape(areas_ground['non_weighted_first'], areas_pred['non_weighted_first'])
mape_non_weight_second=calculate_mape(areas_ground['non_weighted_second'], areas_pred['non_weighted_second'])
mape_weight_first=calculate_mape(areas_ground['weighted_first'], areas_pred['weighted_first'])
mape_weight_second=calculate_mape(areas_ground['weighted_second'], areas_pred['weighted_second'])


#Compute centroids
centroids_pred, centroids_ground=compute_centroids(predictions_test, ground_truth_test)

# Compute centroid distances
distances=compute_centroid_distances(centroids_pred, centroids_ground, image_shape)

#Compute angles
angles=compute_angles(centroids_pred, centroids_ground, image_shape)

#Compute errors for angles
mape_non_weight_first_angle=calculate_mape(angles['non_weighted']['ground'], angles['non_weighted']['pred'])
mape_weight_first_angle=calculate_mape(angles['weighted']['ground'], angles['weighted']['pred'])

nrmse_non_weight_first_angle=calculate_nrmse(angles['non_weighted']['ground'], angles['non_weighted']['pred'])
nrmse_weight_first_angle=calculate_nrmse(angles['weighted']['ground'], angles['weighted']['pred'])


#Calculate NMI and SSIM for a specific region
region_of_interest=(20, 85, 5, 30)
dynamic_roi=determine_dynamic_roi(predictions_test, thresh_zero)

nmi_scores_region=calculate_nmi(predictions_test, ground_truth_test, region=region_of_interest)
ssim_scores_region=calculate_ssim(predictions_test, ground_truth_test, region=region_of_interest)

In [6]:
#Optional: run hyperparameter tuning
study=run_optuna_study(train_dataloader, test_dataloader)